## Unrolling a QASM program
- We take the example from the `openqasm3` repository and produce the unrolled program.  

In [ ]:
%pip install pyqasm --quiet

In [1]:
import pyqasm

pyqasm.__version__

'0.1.0'

In [2]:
from pyqasm import loads, dumps

- [`Adder.qasm`](https://github.com/openqasm/openqasm/blob/9150b157df801702b02ac7c14979ace8aba53a85/examples/adder.qasm)

In [3]:
adder_qasm_str = """
/*
 * quantum ripple-carry adder
 * Cuccaro et al, quant-ph/0410184
 */
OPENQASM 3.0;
include 'stdgates.inc';

gate majority a, b, c {
    cx c, b;
    cx c, a;
    ccx a, b, c;
}

gate unmaj a, b, c {
    ccx a, b, c;
    cx c, a;
    cx a, b;
}

qubit[1] cin;
qubit[4] a;
qubit[4] b;
qubit[1] cout;
bit[5] ans;
array[bool, 4] a_in = {0,0,0,1};  // a = 0001
array[bool, 4] b_in = {1,1,1,1}; // b = 1111
// initialize qubits
reset cin;
reset a;
reset b;
reset cout;

// set input states
for int i in [0: 3] {
  if(a_in[i]) x a[i];
  if(b_in[i]) x b[i];
}

// add a to b, storing result in b
majority cin[0], b[0], a[0];
for int i in [0: 2] { majority a[i], b[i + 1], a[i + 1]; }
cx a[3], cout[0];
for int i in [2: -1: 0] { unmaj a[i],b[i+1],a[i+1]; }
unmaj cin[0], b[0], a[0];
measure b[0:3] -> ans[0:3];
measure cout[0] -> ans[4];
"""

In [4]:
module = loads(adder_qasm_str)

module.validate()

module.unroll()

In [5]:
dumps(module).splitlines()

['OPENQASM 3.0;',
 'include "stdgates.inc";',
 'qubit[1] cin;',
 'qubit[4] a;',
 'qubit[4] b;',
 'qubit[1] cout;',
 'bit[5] ans;',
 'reset cin[0];',
 'reset a[0];',
 'reset a[1];',
 'reset a[2];',
 'reset a[3];',
 'reset b[0];',
 'reset b[1];',
 'reset b[2];',
 'reset b[3];',
 'reset cout[0];',
 'x b[0];',
 'x b[1];',
 'x b[2];',
 'x a[3];',
 'x b[3];',
 'cx a[0], b[0];',
 'cx a[0], cin[0];',
 'ccx cin[0], b[0], a[0];',
 'cx a[1], b[1];',
 'cx a[1], a[0];',
 'ccx a[0], b[1], a[1];',
 'cx a[2], b[2];',
 'cx a[2], a[1];',
 'ccx a[1], b[2], a[2];',
 'cx a[3], b[3];',
 'cx a[3], a[2];',
 'ccx a[2], b[3], a[3];',
 'cx a[3], cout[0];',
 'ccx a[2], b[3], a[3];',
 'cx a[3], a[2];',
 'cx a[2], b[3];',
 'ccx a[1], b[2], a[2];',
 'cx a[2], a[1];',
 'cx a[1], b[2];',
 'ccx a[0], b[1], a[1];',
 'cx a[1], a[0];',
 'cx a[0], b[1];',
 'ccx cin[0], b[0], a[0];',
 'cx a[0], cin[0];',
 'cx cin[0], b[0];',
 'ans[0] = measure b[0];',
 'ans[1] = measure b[1];',
 'ans[2] = measure b[2];',
 'ans[4] = measure 